## Описание проекта

Проект по SQL

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

Таблица `books`

Содержит данные о книгах:
* `book_id` — идентификатор книги;
* `author_id` — идентификатор автора;
* `title` — название книги;
* `num_pages` — количество страниц;
* `publication_date` — дата публикации книги;
* `publisher_id` — идентификатор издателя.

Таблица `authors`

Содержит данные об авторах:
* `author_id` — идентификатор автора;
* `author` — имя автора.

Таблица `publishers`

Содержит данные об издательствах:
* `publisher_id` — идентификатор издательства;
* `publisher` — название издательства;

Таблица `ratings`

Содержит данные о пользовательских оценках книг:
* `rating_id` — идентификатор оценки;
* `book_id` — идентификатор книги;
* `username` — имя пользователя, оставившего оценку;
* `rating` — оценка книги.

Таблица `reviews`

Содержит данные о пользовательских обзорах:
* `review_id` — идентификатор обзора;
* `book_id` — идентификатор книги;
* `username` — имя автора обзора;
* `text` — текст обзора.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры 
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения                      
             'db': 'data-analyst-final-project-db'} # название базы данных 


connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],                                                                         
                                                         db_config['host'],                                                                                     
                                                         db_config['port'],                                                                                     
                                                         db_config['db']) 

# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование таблиц

Выведем 5 сток таблицы books

In [3]:
query = """SELECT * FROM books"""

pd.io.sql.read_sql(query, con = engine).head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Выведем 5 сток таблицы authors

In [4]:
query = """SELECT * FROM authors"""

pd.io.sql.read_sql(query, con = engine).head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Выведем 5 сток таблицы publishers

In [5]:
query = """SELECT * FROM publishers"""

pd.io.sql.read_sql(query, con = engine).head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Выведем 5 сток таблицы ratings

In [6]:
query = """SELECT * FROM ratings"""

pd.io.sql.read_sql(query, con = engine).head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Выведем 5 сток таблицы reviews

In [7]:
query = """SELECT * FROM reviews"""

pd.io.sql.read_sql(query, con = engine).head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Задания


### Посчитаем, сколько книг вышло после 1 января 2000 года

In [8]:
query = """SELECT COUNT(book_id) AS count_books
           FROM books
           WHERE CAST(publication_date AS TIMESTAMP) > '01-01-2000'
           
           """

pd.io.sql.read_sql(query, con = engine).head()

,count_books
0,819


Вывод:
* После 1 января 2000 года вышло 819 книг

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [9]:
query = """WITH tmp AS (SELECT b.book_id,
                               COALESCE(COUNT(review_id), 0) AS count_review
                        FROM books AS b
                            LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
                        GROUP BY b.book_id
                        ),
                        
                        
                 tmp2 AS (SELECT b.book_id,
                                 COALESCE(AVG(rating), 0) AS avg_rate
                          FROM books AS b
                              LEFT JOIN ratings AS rat ON b.book_id = rat.book_id
                          GROUP BY b.book_id
                         )
                        
                        
                SELECT count_review,
                       avg_rate
                FROM tmp
                    JOIN tmp2 ON tmp.book_id = tmp2.book_id
                ORDER BY avg_rate DESC, count_review DESC
           
           
           """

pd.io.sql.read_sql(query, con = engine).head()

,count_review,avg_rate
0,4,5.0
1,3,5.0
2,3,5.0
3,2,5.0
4,2,5.0


Вывод:
* Самые популярные книги по количеству обзоров и рейтингу - это книги A Dirty Job и School's Out—Forever.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [10]:
query = """WITH tmp AS (SELECT publisher_id,
                               COUNT(book_id) AS count_books
                        FROM books
                        WHERE num_pages > 50
                        GROUP BY publisher_id)


            SELECT publisher
            FROM tmp
                JOIN publishers AS p ON tmp.publisher_id = p.publisher_id
            WHERE count_books= (SELECT MAX(count_books) FROM tmp)

           
           """

pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


Вывод:
* Издательство "Penguin Books" выпустило наибольшее число книг толще 50 страниц.

### Определим автора с самой высокой средней оценкой книг — необходимо учитывать только книги с 50 и более оценками

In [11]:
query = """WITH tmp AS (SELECT author_id,
                               b.book_id,
                               AVG(rating) AS avg_rate,
                               COUNT(rating_id) AS count_rate
                        FROM books AS b
                             JOIN ratings AS rat ON b.book_id = rat.book_id
                        GROUP BY author_id, b.book_id
                        HAVING COUNT(rating_id) >= 50
                        )
                        
                        
           SELECT author
           FROM tmp
                JOIN authors AS a ON tmp.author_id = a.author_id
           WHERE avg_rate = (SELECT MAX(avg_rate) FROM tmp)

           """

pd.io.sql.read_sql(query, con = engine)

,author
0,J.K. Rowling/Mary GrandPré


Вывод:
* J.K. Rowling/Mary GrandPré - авторы с самой высокой средней оценкой книг, среди книг с 50 и более оценками.

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = """WITH tmp AS (SELECT username,
                               COUNT(rating_id) AS count_rating
                        FROM ratings
                        GROUP BY username
                        HAVING COUNT(rating_id) > 50)
                        
            
            SELECT ROUND(AVG(count_review), 1) AS avg_review
            FROM (SELECT COUNT(review_id) AS count_review
                  FROM reviews
                  WHERE username IN (SELECT username FROM tmp)
                  GROUP BY username) AS q
        """

pd.io.sql.read_sql(query, con = engine)

,avg_review
0,24.3


Вывод:
* Среди пользователей, которые поставили более 50 оценок, среднее количество обзоров - 24.3